# Class 15:  joint entropy and the REVEAL algorithm

We'll use the bladder cancer gene expression data to test out the REVEAL algorithm. First, we'll load the data and filter to include only genes for which the median log2 expression level is > 14.  That should give us 13 genes to work with (to keep running times reasonable for an in-class notebook).

Import the Python modules that we will need for this exercise

In [1]:
import pandas
import numpy
import itertools

Load the data file `shared/bladder_cancer_genes_tcga.txt` into a `pandas.DataFrame`, convert it to a `numpy.ndarray` matrix, and print the matrix dimensions

In [12]:
gene_matrix_for_network_df = 
gene_matrix_for_network = 


(4473, 414)


Filter the matrix to include only rows for which the column-wise median is > 14; matrix should now be 13 x 414.

In [3]:
genes_keep = 
matrix_filt = 



Binarize the gene expression matrix using the mean value as a breakpoint, turning it into a NxM matrix of booleans (`True`/`False`).  Call it `gene_matrix_binarized`.  Use `numpy.tile` and `numpy.mean` and `transpose`.

In [4]:
gene_matrix_binarized = 


(13, 414)


Test your matrix by printing the first four columns of the first four rows:

array([[False,  True, False, False],
       [False,  True, False, False],
       [ True,  True,  True, False],
       [False,  True, False, False]])

The core part of the REVEAL algorithm is a function that can compute the joint entropy of a collection of binary (TRUE/FALSE) vectors X1, X2, ..., Xn (where length(X1) = length(Xi) = M).
Write a function `entropy_multiple_vecs` that takes as its input a nxM matrix (where n is the number of variables, i.e., genes, and M is the number of samples in which gene expression was measured). The function should use the log2 definition of the Shannon entropy. It should return the joint entropy H(X1, X2, ..., Xn) as a scalar numeric value. I have created a skeleton version of this function for you, in which you can fill in the code. I have also created some test code that you can use to test your function, below.

In [6]:
def entropy_multiple_vecs(binary_vecs):
    ## use shape to get the numbers of rows and columns as [n,M]
    [n, M] = 
    
    # make a "M x n" dataframe from the transpose of the matrix binary_vecs
    binary_df = 
    
    # use the groupby method to obtain a data frame of counts of unique occurrences of the 2^n possible logical states
    binary_df_counts = 
    
    # divide the matrix of counts by M, to get a probability matrix
    probvec = 
    
    # compute the shannon entropy using the formula
    hvec = 
    return 

This test case should produce the value 3.938:

In [7]:
print(entropy_multiple_vecs(gene_matrix_binarized[0:4,]))

3.471945238516729


## Example implementation of the REVEAL algorithm:
We'll go through stage 3

In [8]:
ratio_thresh = 0.1
genes_to_fit = list(range(0,N))
stage = 0
regulators = [None]*N
entropies_for_stages = [None]*N
max_stage = 4

entropies_for_stages[0] = numpy.zeros(N)

for i in range(0,N):
    single_row_matrix = gene_matrix_binarized[i,:,None].transpose()
    entropies_for_stages[0][i] = entropy_multiple_vecs(single_row_matrix)
    
genes_to_fit = set(range(0,N))

In [10]:
for stage in range(1,max_stage + 1):
    for gene in genes_to_fit.copy():
        # we are trying to find regulators for gene "gene"
        poss_regs = set(range(0,N)) - set([gene])
        poss_regs_combs = [list(x) for x in itertools.combinations(poss_regs, stage)]
        HGX = numpy.array([ entropy_multiple_vecs(gene_matrix_binarized[[gene] + poss_regs_comb,:]) for poss_regs_comb in poss_regs_combs ])
        HX = numpy.array([ entropy_multiple_vecs(gene_matrix_binarized[poss_regs_comb,:]) for poss_regs_comb in poss_regs_combs ])
        HG = entropies_for_stages[0][gene]
        min_value = numpy.min(HGX - HX)
        if HG - min_value >= ratio_thresh * HG:
            regulators[gene]=poss_regs_combs[numpy.argmin(HGX - HX)]
            genes_to_fit.remove(gene)
results = [[index, value] for index, value in enumerate(regulators)]

In [11]:
results

[[0, [1]],
 [1, [0]],
 [2, [8]],
 [3, [1, 2, 4, 6]],
 [4, [0, 3, 5, 11]],
 [5, [1, 2, 6, 10]],
 [6, [7]],
 [7, [6]],
 [8, [2]],
 [9, [7, 8, 10]],
 [10, [11]],
 [11, [10]],
 [12, [1, 6, 9]]]